<a href="https://colab.research.google.com/github/Jacob-Brink/.emacs.d/blob/linodeRemote/Homework1_Data_FINALLY_LOADS_UGGGGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 1

In [49]:
# For Colab, add (and run) this code cell above everything else 

try: import fastbook 

except ImportError: import subprocess; subprocess.run(['pip','install','-Uq','fastbook']) 

In [69]:
# Import yelp data set from our shared google drive
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [71]:
DATA_PATH = "/content/gdrive/Shareddrives/CS344-DATA/homework-01/yelp_photos_subset001/"

In [91]:
get_files??

import json

# reduces yelp files by only returning the paths of files that match the given list of labels
def prune_yelp_photos(path_list, sorted, labels):

  pruned_files = []

  for path in path_list:
    label = sorted[(path.name).split(".")[0]]
    if (label in labels):
      pruned_files.append(path)
  
  return L(pruned_files)


# sorting the data, keys are the id of the photo and the value is the label the photo has
sorted_data = {}

# label function
def is_inside(file_name):
  return sorted_data[file_name.split(".")[0]] == "inside"

with open(f"{DATA_PATH}photos.json") as file:
  # load data
  data = json.load(file) 

  # sort through data
  for p in data:
    photo_label = p['label']
    photo_file_name = p['photo_id']
    sorted_data[photo_file_name] = photo_label




In [ ]:
#id first_training
#caption Results from the first training
# CLICK ME
from fastai.vision.all import * # imports fastai

def is_cat(x): return x[0].isupper() # determines label of data from filename
dls = ImageDataLoaders.from_name_func(
    DATA_PATH, prune_yelp_photos(get_files(f"{DATA_PATH}photos"), sorted_data, ['inside', 'outside']), valid_pct=0.2, seed=42,
    label_func=is_inside, item_tfms=Resize(224)) # sets up ai

learn = cnn_learner(dls, resnet34, metrics=error_rate) 
learn.fine_tune(1)


Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


epoch,train_loss,valid_loss,error_rate,time
0,0.891092,0.183326,0.064615,08:36


epoch,train_loss,valid_loss,error_rate,time


In [ ]:
ImageDataLoaders.from_name_func?

**Question**: Use `learn.fine_tune?` to bring up the documentation. Based on that:

1. What does the "1" parameter to `fine_tune` do?
2. What do you think the two different progress bars probably represent?

In [ ]:
1. The number 1 tells the AI to look at each image once.
2. The second bar is the training data, and the first bar is the 

**Question**: Give an example filename for one of the images in the dataset. Does the dataset label it as a cat or a dog? How can you tell?

In [ ]:
from os import listdir
print(listdir(path)[0])

# this is a dog, because it is a filename that starts with a lowercase letter

**Question**: How many images were in the training set? Validation set? (How can you easily tell?)

In [ ]:
# your code here
print(Counter(
    get_image_files(path)
    .attrgot('name')
    .map(true)
))

print(f'Images in Validation set: {7390*.2}')
print(f'Images in Training set: {7390*.8}')

*Your response here. You may need to write a tiny bit of code.*

**Question**: About how many of those images were classified correctly?

In [ ]:
print(Counter(
    get_image_files(path)
    .attrgot('name')
    .map(is_cat)
))

*** HELP ***

**Question**: What is the output of the model?

1. whether the image contains one cat (`True`) or many cats (`False`)
2. whether the image contains a cat (`True`) or a dog (`False`)
3. what breed of cat or dog is in the image (`British_Shorthair`, `chihuahua`, etc.)
4. the name of the animal in the image (`Fido`, `Whiskers`, etc.)

**Exercise**: Use an 18-layer ResNet instead (`resnet18`). How does the error rate compare?

In [ ]:
from fastai.vision.all import * # imports fastai
path = untar_data(URLs.PETS)/'images' # gathers the data

def is_cat(x): return x[0].isupper() # determines label of data from filename
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_cat, item_tfms=Resize(224)) # sets up ai

learn = cnn_learner(dls, resnet18, metrics=error_rate) 
learn.fine_tune(1)

The error rate is worse when we used 18 layers by a bit.

Consider the We just changed the *architecture*, but not anything else. We could also change:

* The *data*: give it different images (or more or fewer of the same images)
* The *task*: have it try to predict something different
* The *hyperparameters*: train it a different way, e.g., let it train longer (more epochs) or adjust itself faster (higher learning rate)

Let's change the data.

**Exercise**: Hold out 90% (instead of 20%) of the data for validation. (How many images will the training set have now?) How does the accuracy compare?

(*Think*: Should you use the resnet18 or the resnet34 model here? What are the pros and cons?)

In [ ]:
from fastai.vision.all import * # imports fastai
path = untar_data(URLs.PETS)/'images' # gathers the data

def is_cat(x): return x[0].isupper() # determines label of data from filename
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.9, seed=42,
    label_func=is_cat, item_tfms=Resize(224)) # sets up ai

learn = cnn_learner(dls, resnet18, metrics=error_rate) 
learn.fine_tune(1)

The accuracy gets much worse than the resnet18 with 20% of the data used for validation.

**Question**: How might we change the *task*? That is, what different things could we have the model output?

Think about that for a moment.


Which ones are male and which ones are female.

.

.

The model's outputs are (generally) determined by what sort of *labels* we have for our data. Let's have a look at what the files are named. We could open up our data directory, or we could look in Python.

In [ ]:
[o.name for o in get_image_files(path)][:10] # show only the first 10

In [ ]:
# Aside: fastai added a nice trick that makes this easy:
get_image_files(path).attrgot('name')

So it looks like we should be able to get the *breed* pretty easily. All we need to do is strip off everything after the last underscore. (Is this exercise familiar?)

These names are passed to the `name_func`. So all we need to do is strip off the number and file extension at the end to get the breed.

**Exercise**: Write a function called `get_breed` that takes a string and strips off everything after the last underscore.

In [ ]:
def get_breed(x):
    return '_'.join(x.split("_")[0:-1])

Now let's test that function.

In [ ]:
# When a Python expression gets complicated, I like to spread it onto multiple lines.
# Often an expression is already inside some parentheses so you can just press Enter,
# but for "top-level" expressions we need this trick of surrounding the whole thing in parens.
# Aside from the attrot and map, this is just standard Python here, no special magic.
(
    get_image_files(path)
    .attrgot('name')
    .map(get_breed)
)

Let's check how many of each breed we have.

In [ ]:
Counter(
    get_image_files(path)
    .attrgot('name')
    .map(get_breed)
)

Now, let's change the model to output the breed instead of whether or not the thing is a cat.

**Predict**: Will the accuracy number go up or down? Why?

A: It will go down, because for each classification there will be less training data. With the cat vs dog, about half of the training data went for each classification, but now  less than 10% of the training data is used for each classification.

**Exercise**: Go back to the initial setup (ResNet-34 model and full dataset), but instead of predicting cat vs dog, now predict breed.

In [ ]:
from fastai.vision.all import * # imports fastai
path = untar_data(URLs.PETS)/'images' # gathers the data

dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42,
    label_func=get_breed, item_tfms=Resize(224)) # sets up ai

learn = cnn_learner(dls, resnet34, metrics=error_rate) 
learn.fine_tune(1)

**Reflect**: What accuracy do you get with this classifier? Is it better or worse? How can you tell?

A: The accuracy is really bad, compared to the previous models, since  the error rate is between 9 and 12%.

# Open-ended!

If you got through that quickly, now try one or two more experiments of your own choice. For example, you might try changing a hyperparameter (epochs, learning rate, ...), try a different dataset size, try different random seeds, ...

# Summarize

1. Write a short summary of the results of all of the experiments you performed in this lab.

We learned that the amount of training data, the amount of classifications made by the AI, and the amount of layers in the neural network are all important parameters needed to make a good AI. 

2. Reflect on how confident you are that each of those comparisons didn't just happen by chance. Also reflect on which of the results would be likely to generalize to other tasks.

I think it is highly unlikely that the comparisons happened by chance. I think most classification problems would be doable with this approach.


# Wrap-up

* Make sure you Save this notebook.
* From the Kernel menu, select "Restart and Run All". This will take a few minutes, but will ensure that all of your code still runs. Double-check that your results still make sense.
* Commit and push. Check that your notebook is visible on GitHub.